### Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. 

Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
- Наименование вакансии.
- Предлагаемую зарплату (отдельно минимальную и максимальную).
- Ссылку на саму вакансию.
- Сайт, откуда собрана вакансия.


По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

На примере сайта Superjob:

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [2]:
def get_html(url, params):
    response = requests.get(url, params=params).text
    html = bs(response, 'html.parser')
    
    return html

In [3]:
def get_last_page(html):
    pagination = html.find('a', {'class': 'f-test-button-1'})
    if not pagination:
        last_page = 1
    else:
        pagination = pagination.findParent()
        last_page = int(pagination.find_all('a')[-2].text)
    
    return last_page

In [4]:
def get_vacancy_details(vacancy):
    vacancy_details = {'city': [],
                       'name': [],
                       'company': [],
                       'salary_min': [],
                       'salary_max': [],
                       'link': [], 
                       'site': []
                      }
    # city
    vacancy_details['city'] = vacancy.find(attrs={'class': 'f-test-text-company-item-location'})\
    .findChildren()[2].text.split(',')[0]

    # job title
    vacancy_details['name'] = vacancy.find('a', href=True).text

    # job link
    vacancy_details['link'] = 'https://www.superjob.ru' + vacancy.find('a', href=True)['href']

    # company name
    vacancy_details['company'] = vacancy.find(attrs={'class': 'f-test-text-vacancy-item-company-name'}).text if \
        vacancy.find(attrs={'class': 'f-test-text-vacancy-item-company-name'}) else None

    # job site
    vacancy_details['site'] = 'Superjob'

    # salary
    salary = vacancy.find(attrs={'class': 'f-test-text-company-item-salary'}).next.text
    salary = re.split(r'\s|-', salary)
    if salary[0] == 'до':
        salary_min = None
        salary_max = int(salary[1]+salary[2])
    elif salary[0] == 'от':
        salary_min = int(salary[1]+salary[2])
        salary_max = None
    elif salary[0] == 'По':
        salary_min = None
        salary_max = None
    else:
        salary_min = int(salary[0]+salary[1])
        salary_max = int(salary[3]+salary[4]) if len(salary) > 3 else salary_min
    vacancy_details['salary_min'] = salary_min
    vacancy_details['salary_max'] = salary_max
    
    return vacancy_details

In [5]:
def get_vacancies(jobname):
    vacancies_list = []
    url = 'https://russia.superjob.ru/vacancy/search/'
    params = {
        'keywords': jobname,
        'profession_only': 1,
        'page': ''
    }
    html = get_html(url, params)
    for page in range(1, get_last_page(html) + 1):
        params['page'] = page
        get_html(url, params)
        if html.find_all(attrs={'class': 'f-test-vacancy-item'}):
            results = html.find_all(attrs={'class': 'f-test-vacancy-item'})
        for result in results:
            result = get_vacancy_details(result)
            vacancies_list.append(result)
    
    return vacancies_list

In [6]:
vacancies = get_vacancies('Тестировщик')
df = pd.DataFrame(vacancies)
df

,city,name,company,salary_min,salary_max,link,site
0,Санкт-Петербург,Тестировщик ПО,Сбербанк России,NaN,NaN,https://www.superjob.ru/vakansii/testirovschik...,Superjob
1,Москва,Тестировщик,"Предприятие ""АЛИДИ""",80000.0,90000.0,https://www.superjob.ru/vakansii/testirovschik...,Superjob
2,Москва,Системный аналитик / Тестировщик,Российские космические системы,NaN,NaN,https://www.superjob.ru/vakansii/sistemnyj-ana...,Superjob
3,Москва,Тестировщик ПО и радиоустройств,Смартико,NaN,105000.0,https://www.superjob.ru/vakansii/testirovschik...,Superjob
4,Москва,Тестировщик ПО,Делаем бизнес,NaN,120000.0,https://www.superjob.ru/vakansii/testirovschik...,Superjob
...,...,...,...,...,...,...,...
115,Санкт-Петербург,Тестировщик,Нинтегра,50000.0,NaN,https://www.superjob.ru/vakansii/testirovschik...,Superjob
116,Орехово-Зуево,"Технический писатель, тестировщик",НВП Болид,45000.0,NaN,https://www.superjob.ru/vakansii/tehnicheskij-...,Superjob
117,Нижний Новгород,Специалист по тестированию ПО / Тестировщик ПО,Максавит,60000.0,60000.0,https://www.superjob.ru/vakansii/specialist-po...,Superjob
118,Москва,QA engineer / Тестировщик,СтандартПроект,NaN,NaN,https://www.superjob.ru/vakansii/qa-engineer-3...,Superjob
